## Data

I think I need to figure out what I want to do with this first...

Probably start off with a model that predicts an ingredient list.

So I need to clean the ingredient data.

In [19]:
%load_ext autoreload
%autoreload 2

In [1]:
cd /home/ec2-user/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [16]:
from src.data.pickling import load_pickle
import copy

In [3]:
ls data/raw/

shoestring_recipes.pickle  test.pickle


In [14]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [17]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [20]:
list(shoestring_recipes.keys())[:5]

['https://glutenfreeonashoestring.com/d-y-friday-homemade-hot-chocolate-mix/',
 'https://glutenfreeonashoestring.com/gluten-free-carrot-cake-cupcakes/',
 'https://glutenfreeonashoestring.com/gluten-free-strawberry-shortcake/',
 'https://glutenfreeonashoestring.com/',
 'https://glutenfreeonashoestring.com/vegan-gluten-free-muffins/']

In [ ]:
for recipe in shoestring_recipes:
    print(list(shoestring_recipes[recipe].values())[0])

In [49]:
urls = shoestring_recipes.keys()

In [101]:
recipe_dict = {}

for recipe in shoestring_recipes:
    try:
        for key in shoestring_recipes[recipe].keys():
            recipe_dict[key] = shoestring_recipes[recipe][key]["recipeIngredient"]
    except:
        pass

In [ ]:
recipe_dict

In [ ]:
measurement = [
    "cup",
    "teaspoon",
    "ounce",
    "tablespoon",
    "pinch",
    "tsp",
    "tbsp"
]

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [9]:
# nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [78]:
def pos_tagging(words):
    tokenized = nltk.word_tokenize(words)
    tokens_pos = nltk.pos_tag(tokenized)
    return tokens_pos

In [12]:
# add numbers together
from fractions import Fraction
sum_fractions = Fraction(1/4) + Fraction(2)
# clean non words and numbers

In [13]:
# for single recipe and single
ingredients_pos = []

for ingredient in shoestring_recipes[
    'https://glutenfreeonashoestring.com/gluten-free-carrot-cake-cupcakes/'
]['Fluffy, Tender, and Easy Gluten Free Carrot Cake (or Cupcakes!)']["recipeIngredient"]:
    ingredients_pos.append(pos_tagging(ingredient))

ingredient = ingredients_pos[0]
remove_index = []
final_nbr = 0
sum_nbr = 0

for item in ingredient:
    if item[1] == "CD":
        remove_index.append(item)
        final_nbr += Fraction(item[0])
    else:
        # add sum to beginning of that ingredient
        sum_nbr = round(final_nbr.numerator / final_nbr.denominator, 2)
        break

# remove it
for remove in remove_index:
    ingredient.remove(remove)

new_ingredient = [(sum_nbr, "CD")] + ingredient

print(new_ingredient)

[(2.25, 'CD'), ('cups', 'NNS'), ('all', 'DT'), ('purpose', 'VBP'), ('gluten', 'NNS'), ('free', 'JJ'), ('flour', 'NNS'), ('blend', 'VBP')]


In [76]:
list(recipe_dict.values())[0]

['3/4 cup unsweetened cocoa powder ((preferably Dutch-processed))',
 '1/2 cup powdered milk',
 '2 teaspoons gluten free sweet rice flour',
 '1/2 teaspoon kosher salt',
 '3/4 cup granulated sugar',
 '2 ounces bittersweet chocolate, at cool room temperature (finely grated or pulsed in blender or food processor (optional))',
 '1/3 cup powdered milk',
 '1 1/2 teaspoons gluten free sweet rice flour',
 '1/2 teaspoon kosher salt',
 '1/4 cup granulated sugar',
 '4 ounces white chocolate, at cool room temperature (finely grated or pulsed in blender or food processor)',
 '3 tablespoons dry mix',
 '1 cup milk',
 'Miniature marshmallows (for serving (optional))']

In [131]:
# for single recipe and single 
recipe_dict_test = copy.deepcopy(recipe_dict)
ingredients_pos = []

for ingredient in list(recipe_dict_test.values())[0]:
    ingredients_pos.append(pos_tagging(ingredient))

In [197]:
list(recipe_dict_test.values())[1]

['2 1/4 cups all purpose gluten free flour blend',
 '1 teaspoon xanthan gum (omit if your blend already contains it)',
 '1 teaspoon baking soda',
 '1 1/2 teaspoons baking powder',
 '1/2 teaspoon kosher salt',
 '2 teaspoons ground cinnamon',
 '3/4 cup granulated sugar',
 '1/2 cup packed light brown sugar',
 '1/2 cup neutral oil ((like sunflower or grapeseed oil))',
 '4  eggs (at room temperature, beaten)',
 '1 teaspoon apple cider vinegar',
 '1/2 cup milk (at room temperature)',
 '1/2 cup chopped pecans (plus more for decorating (optional))',
 '1/2 cup white chocolate chips ((See Recipe Notes if omitting pecans))',
 '3 cups peeled and grated carrots ((from about 3 large carrots))',
 '8 ounces cream cheese (at room temperature)',
 '8 tablespoons unsalted butter ( at room temperature)',
 '1/4 teaspoon kosher salt',
 '4 cups confectioners’ sugar (plus more as necessary)']

In [242]:
# for single recipe and single 
recipe_dict_test = copy.deepcopy(recipe_dict)
ingredients_pos = []

# POS tagging for ingredients list
for ingredient in list(recipe_dict_test.values())[1]:
    ingredients_pos.append(pos_tagging(ingredient))

# # make smaller so easier to comprehend
# ingredients_pos = ingredients_pos

# For each ingredient in an ingredients list
for ingredient in range(0,len(ingredients_pos)):
    remove_index = []
    final_nbr = 0
    sum_nbr = 0
    nbr_cd = 0
    # For each POS of an ingredient
    for pos in ingredients_pos[ingredient]:
        # If it's a quantity
        if pos[1] == "CD":
            # Track number of POS quantities related to this ingredient
            nbr_cd += 1
            # Get index of POS
            pos_index = ingredients_pos[ingredient].index(pos)
            # Turn quantity to a real number
            final_nbr += Fraction(pos[0])
            round_nbr = round(
                final_nbr.numerator / final_nbr.denominator, 2
            )
            new_pos = (round_nbr, "CD")
            # Set old POS as new
            ingredients_pos[ingredient][pos_index] = new_pos
            # If multiple quantity POS's related to ingredient
            if nbr_cd > 1:
                # Remove previous quantity
                print(ingredients_pos[ingredient][pos_index-1])
                ingredients_pos[ingredient].pop(pos_index-1)

(2.0, 'CD')
(1.0, 'CD')
('about', 'IN')


In [ ]:
tokenized = nltk.word_tokenize(shoestring_recipes[
    'https://glutenfreeonashoestring.com/gluten-free-carrot-cake-cupcakes/'
]['Fluffy, Tender, and Easy Gluten Free Carrot Cake (or Cupcakes!)'
 ]["recipeIngredient"][0])

nltk.pos_tag(tokenized)

# Next steps

- Clean recipe data (remove everything after a parenthesis)
- If "or", choose one option (e.g., "1 cup butterscotch or peanut butter chips")
- Remove untagged words

POS tag list:

POS_tag | definiton
- | -
CC | coordinating conjunction
CD | cardinal digit
DT | determiner
EX | existential there (like: "there is" ... think of it like "there exists")
FW | foreign word
IN | preposition/subordinating conjunction
JJ | adjective 'big'
JJR | adjective, comparative 'bigger'
JJS | adjective, superlative 'biggest'
LS | list marker 1)
MD | modal could, will
NN | noun, singular 'desk'
NNS | noun plural 'desks'
NNP | proper noun, singular 'Harrison'
NNPS | proper noun, plural 'Americans'
PDT | predeterminer 'all the kids'
POS | possessive ending parent's
PRP | personal pronoun I, he, she
PRP$ | possessive pronoun my, his, hers
RB | adverb very, silently,
RBR | adverb, comparative better
RBS | adverb, superlative best
RP | particle give up
TO | to go 'to' the store.
UH | interjection errrrrrrrm
VB | verb, base form take
VBD | verb, past tense took
VBG | verb, gerund/present participle taking
VBN | verb, past participle taken
VBP | verb, sing. present, non-3d take
VBZ | verb, 3rd person sing. present takes
WDT | wh-determiner which
WP | wh-pronoun who, what
WP$ | possessive wh-pronoun whose
WRB | wh-abverb where, when